## TF-IDF from scratch in Python

Concepts explore here are:

 1. TFIDF.
 2. Similarity search given a **Query** and TFIDF weights.
 3. Similarity search with **Cosine similarity** given a Query.

In [1]:
import os, pickle

import nltk, string, copy, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import numpy as np, pandas as pd
import math

In [2]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [3]:
documents = ["An apple a day keeps the doctor away.", "Never compare an apple to an orange.",
 "I prefer scikit-learn to orange"]
documents

['An apple a day keeps the doctor away.',
 'Never compare an apple to an orange.',
 'I prefer scikit-learn to orange']

In [4]:
processed_docs = []
[processed_docs.append(word_tokenize(str(preprocess(text)))) for text in documents]
processed_docs

In [5]:
DF = {token:0 for doc in processed_docs for token in doc}
vocab = list(DF.keys())
for doc in processed_docs:
    for token in DF.keys():
        if token in doc:
            DF[token] +=1
            
vocab_size = len(vocab)
vocab_size

11

In [6]:
N = len(processed_docs) # Total number of documents
tfidf = {}

for i, doc in enumerate(processed_docs):
    counter = Counter(doc)
    words_count = len(doc)
    
    for token in set(doc):
        tf = counter[token] / words_count
        idf = np.log(N / DF[token] + 1)
        tfidf[i, token] = tf * idf
        
tfidf

{(0, 'day'): 0.2772588722239781,
 (0, 'doctor'): 0.2772588722239781,
 (0, 'keep'): 0.2772588722239781,
 (0, 'away'): 0.2772588722239781,
 (0, 'appl'): 0.18325814637483104,
 (1, 'never'): 0.34657359027997264,
 (1, 'appl'): 0.22907268296853878,
 (1, 'compar'): 0.34657359027997264,
 (1, 'orang'): 0.22907268296853878,
 (2, 'orang'): 0.22907268296853878,
 (2, 'prefer'): 0.34657359027997264,
 (2, 'scikit'): 0.34657359027997264,
 (2, 'learn'): 0.34657359027997264}

### Ranking using Matching Score

In [7]:
query = "I'd like an apple."
query_tokens = word_tokenize(str(preprocess(query)))

query_weights = {}

for k, v in tfidf.items():
    if k[1] in query_tokens:
        if k[0] in query_weights:
            query_weights[k[0]] += v
        else: query_weights[k[0]] = v

In [8]:
query_weights

{0: 0.18325814637483104, 1: 0.22907268296853878}

In [9]:
documents[sorted(query_weights)[-1]]

'Never compare an apple to an orange.'

### Ranking using Cosine Similarity

**Matching score** gives relevant documents but quite fails when we give **long query**.

**Cosine similarity** will rank all documents as vectors of **TFIDF** tokens and consider the **angle** between the two vectors.

In [10]:
def compute_cosine_similarity(a, b):
    '''compute consine similarity between two vectors'''
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [11]:
D = np.zeros((N, vocab_size))
D

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
for k, v in tfidf.items():
    ind = vocab.index(k[1])
    D[k[0]][ind] = v
    
D

array([[0.18325815, 0.27725887, 0.27725887, 0.27725887, 0.27725887,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.22907268, 0.        , 0.        , 0.        , 0.        ,
        0.34657359, 0.34657359, 0.22907268, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.22907268, 0.34657359, 0.34657359,
        0.34657359]])

We will add the query to the set of documents and compute the query tfidf score, then generates the query vector for comparison with the cosine similarity.

In [13]:
Q = np.zeros((vocab_size))
Q

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
query_tokens = word_tokenize(str(preprocess(query)))

counter = Counter(query_tokens)
words_count = len(query_tokens)

for i, token in enumerate(set(query_tokens)):
    tf = counter[token] / words_count
    df = DF[token] if token in vocab else 0
    idf = np.log((N+1) / (df + 1))
    Q[i] = tf * idf
    
Q

array([0.46209812, 0.09589402, 0.46209812, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [15]:
cosine_similarity = {i:0 for i in range(len(D))}
for i, doc_vector in enumerate(D):
    cosine_similarity[i] = compute_cosine_similarity(Q, doc_vector)

In [16]:
cosine_similarity

{0: 0.6205968869162394, 1: 0.2727800388062073, 2: 0.0}

In [17]:
documents[sorted(cosine_similarity)[0]]

'An apple a day keeps the doctor away.'